# Рекомендаційні системи (Recommender Systems)

| Користувач / Фільм 🎬 | Фільм A | Фільм B | Фільм C | Фільм D |
| :-------------------: | :-----: | :-----: | :-----: | :-----: |
|       👤 **U1**       |    5    |    4    |    ?    |    2    |
|       👤 **U2**       |    3    |    ?    |    4    |    1    |
|       👤 **U3**       |    ?    |    2    |    5    |    4    |
|       👤 **U4**       |    4    |    3    |    ?    |    ?    |


## Колаборативна фільтрація (collaborative filtering)

Колаборативна фільтрація (CF) — це підхід у системах рекомендацій, який прогнозує,
які об’єкти (товари, фільми, книги тощо) можуть сподобатися користувачу,
виходячи з подібності між користувачами або між самими об’єктами.

**Підходи «користувач – користувач»**: знайти користувачів, які найбільш схожі на мене (на основі 
тільки тих елементів, які оцінені для нас обох), і передбачити оцінки для інших 
елементів на основі середнього значення 

**Підходи «елемент – елемент»**: знайти елементи, найбільш схожі на даний елемент (на основі 
всіх користувачів, які оцінили обидва елементи), і передбачити оцінки для інших користувачів на основі 
середнього значення

**Підходи матричної факторизації**: знайти деяку низькорангову декомпозицію матриці $X$,
 яка узгоджується з спостережуваними значеннями


Маємо:

- множину користувачів $ U = {u_1, u_2, \ldots, u_m} $,
- множину об’єктів $I = {i_1, i_2, \ldots, i_n} $.

Створюється матриця оцінок:

$$
R = [r_{ui}]_{m \times n}
$$

де:

- $r_{ui}$ — рейтинг, який користувач $ u $ поставив об’єкту $ i $,
- якщо користувач не оцінив об’єкт — $ r_{ui} $ відсутній (невідомий).


Передбачити невідомий рейтинг:
$$
\hat{r_{ui}} \approx r_{ui}
$$

для пари *(користувач, об’єкт)*, якої ще немає у матриці.

**User-based Collaborative Filtering (на основі подібності користувачів)**

Ідея:
Шукаємо користувачів, схожих на користувача $u $, і прогнозуємо його рейтинг
на основі оцінок цих схожих користувачів.

*Кроки*:

1. Обчислити схожість між користувачами, наприклад, за допомогою косинусної подібності:
   $$
   \text{sim}(u, v) = \frac{\sum_{i \in I_{uv}} r_{ui} r_{vi}}{\sqrt{\sum_{i \in I_{uv}} r_{ui}^2} \sqrt{\sum_{i \in I_{uv}} r_{vi}^2}}
   $$
   де $ I_{uv} $ — множина об’єктів, які оцінили обидва користувачі.

2. Прогноз рейтингу:
   $$
   \hat{r_{ui}} = \bar{r_u} + \frac{\sum_{v \in N(u)} \text{sim}(u, v) (r_{vi} - \bar{r}_v)}{\sum_{v \in N(u)} |\text{sim}(u, v)|}
   $$
   де:

   - $ N(u) $ — набір найбільш схожих користувачів (сусідів),
   - $\bar{r}_u $ — середній рейтинг користувача $u$.



Окремо можна винести: **Кореляцію Пірсона**:
  $$
  \text{sim}(x, y) = \frac{\sum_i (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_i (x_i - \bar{x})^2} \sqrt{\sum_i (y_i - \bar{y})^2}}
  $$

In [2]:
import numpy as np
import pandas as pd

# Example user-item matrix with ratings
ratings_matrix = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])

# Convert the matrix to a DataFrame for easier handling
ratings_df = pd.DataFrame(ratings_matrix, columns=["Item1", "Item2", "Item3", "Item4"])

user_similarity = ratings_df.T.corr(method='pearson')

print("User-User Pearson Correlation Matrix:\n", user_similarity)

User-User Pearson Correlation Matrix:
           0         1         2         3         4
0  1.000000  0.774291 -0.186441 -0.178683 -0.978839
1  0.774291  1.000000  0.019854  0.162791 -0.628768
2 -0.186441  0.019854  1.000000  0.972828  0.221028
3 -0.178683  0.162791  0.972828  1.000000  0.258904
4 -0.978839 -0.628768  0.221028  0.258904  1.000000


In [3]:
# Predict function to compute weighted ratings
def predict_rating(user_index, item_index, ratings_df, similarity_matrix):
    user_means = ratings_df.replace(0, np.nan).mean(axis=1).values

    # Get the ratings for the item from all other users
    item_ratings = ratings_df.iloc[:, item_index]
    
    # Get the similarity of the target user with all other users
    user_similarities = similarity_matrix[user_index]
    
    # Filter for only users who rated the item and exclude self-similarity
    non_zero_indices = item_ratings[item_ratings > 0].index
    similarities = user_similarities[non_zero_indices]
    ratings = item_ratings[non_zero_indices]
    means = user_means[non_zero_indices]
    
    # If no other user has rated this item, return 0 as prediction
    if len(similarities) == 0:
        return 0
    
    # Weighted sum of ratings and normalize by similarity sum
    
    ratings_weighted = ratings - means
    weighted_sum = np.dot(similarities, ratings_weighted)
    
    sum_of_similarities = np.sum(np.abs(similarities))

    predicted = weighted_sum / sum_of_similarities if sum_of_similarities != 0 else 0

    return ratings.mean() + predicted

print(predict_rating(4, 0, ratings_df, user_similarity))

1.0331985858244794
